In [ ]:
import numpy as np
import tensorflow as tf
import os

In [ ]:
# Text file data converted to integer data type
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/Colab\ Notebooks/AI_project


Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/AI_project


In [ ]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/AI_project


In [ ]:
#Load txt file
def getRawData(string):
  lines = []

  f = open(string, "r")

  lines = f.readlines()

  return lines

In [ ]:
#turns [0,0,0,1] into 4 for directions zero default 
def getDir(line):
	if(line[0] == '1'):
		return 1
	if(line[1] == '1'):
		return 2
	if(line[3] == '1'):
		return 3
	if(line[4] == '1'):
		return 4
	return 0 

In [ ]:
def genMatricies(lines):
	x_data = []
	x_row_list = []
	y_data = []

	for x in range(0,len(lines)):
		# This is used to get rid of the \n in the txt file
		if (x % 12 == 0):
			continue
		#/the 51th line is the direction vector
		if (x % 12 == 11):
			y_data.append(getDir(lines[x]))
			
		#everything else is the matrix
		else:
			#get each char we then convert it to string for tensorflow calcs
			for sym in lines[x]:
				i = 0
				if(sym == 'X'):
					i = -1
				elif(sym == '\n'):
					continue
				elif(sym == '2'):
					i = 2
				elif(sym == '1'):
					i = 1
				x_row_list.append(i)
			#when we have 2500 chars we know we have a 50x50 make it an array and add it to list
			if(len(x_row_list) == 100):
				x_data.append(np.array(x_row_list).reshape(10,10))
				x_row_list = []
			# turn list into numpy array, turn numpy array into catagorical input
	return (np.array(x_data),tf.keras.utils.to_categorical(np.array(y_data), 4))

DATA CREATION

In [ ]:
lines = getRawData("proto10_train.txt")
x1 ,y1 = genMatricies(lines)

In [ ]:
np.save('proto10_train_x1.npy', x1)


In [ ]:
np.save('proto10_train_y1.npy', y1)

In [ ]:
x1 = np.load("proto10_train_x1.npy")
y1 = np.load("proto10_train_y1.npy")

In [ ]:
#train_x1 = np.load('proto50_500_train_x1.npy') 
#train_y1 = np.load('proto50_train_y1.npy')

In [ ]:
#train_y1 = np.load('proto50_500_train_y1.npy')
#train_y2 = np.load('proto50_train_y2.npy')
#total_y = np.append(train_y1, train_y2)
#np.save('proto50_train_total_y.npy', total_y)

TRAINING AGENT

In [ ]:
print(tf.config.list_physical_devices('GPU'))
#checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto50_model/proto50_train"
checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/CNN/"
checkpoint_dir = os.path.dirname(checkpoint_path)
EPOCHS_DEFAULT = 1

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

def buildModel():
  global EPOCHS_DEFAULT

  model = Sequential()
  
  #add model layers
  model.add(Conv2D(16, kernel_size=3, activation= tf.nn.relu, input_shape=(10,10,1)))
  #model.add(Conv2D(4, kernel_size=3, activation= tf.nn.relu))
  model.add(Flatten())
  model.add(Dense(4, activation= tf.nn.softmax))

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model


In [ ]:
#train the model
model = buildModel()


#model.fit(x1, y1, validation_data=(X_test, y_test), epochs=3)

In [ ]:
# this allows you to resume or expand upon training
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
#tf.test.is_gpu_available()

In [ ]:
model = buildModel()

In [ ]:
#model = buildModel()

#model = tf.keras.models.load_model(checkpoint_path)

history = model.fit(x1 , y1, epochs = EPOCHS_DEFAULT,
          callbacks=[cp_callback] )


26979/26983 [============================>.] - ETA: 0s - loss: 0.2683 - accuracy: 0.9065
Epoch 00001: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/CNN/
26983/26983 [==============================] - 75s 3ms/step - loss: 0.2683 - accuracy: 0.9065


TESTING

In [ ]:
def getTestData(string):
  lines = []

  f = open(string, "r")

  lines = f.readlines()

  return lines

In [ ]:
test_lines = getTestData("proto10_test.txt")

tx,ty = genMatricies(test_lines)

In [ ]:
np.save('proto10_test_x1.npy', tx)
np.save('proto10_test_y1.npy', ty)

In [ ]:
tx = np.load('proto10_test_x1.npy')
ty = np.load('proto10_test_y1.npy')

In [ ]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

model.fit(x1, y1, validation_data=(tx, ty), epochs=2)

Epoch 1/2
26983/26983 [==============================] - 87s 3ms/step - loss: 0.1537 - accuracy: 0.9520 - val_loss: 0.1812 - val_accuracy: 0.9462
Epoch 2/2
26983/26983 [==============================] - 86s 3ms/step - loss: 0.1397 - accuracy: 0.9560 - val_loss: 0.1699 - val_accuracy: 0.9505


GEN MAZE

In [ ]:
#map constructor
from random import randint
from matplotlib import pyplot as plot


In [ ]:
#size of each grid
grid_size = 10
#dim x dim for maze
size = int(10)
#prob of wall appearance (0~1)
p_wall = 30

In [ ]:
maze= [['w' for _ in range(size)]for _ in range(size)]

In [ ]:
def createMaze():

  x_data = []
  x_row_list = []

  for row in range(size):
      for col in range(size):
          s = randint(1,100)
          p = p_wall
          if s<=p:
            maze[row][col] = 'X'
          else:
            maze[row][col] = '0'
            
  maze[0][0]= '0'
  maze[size-1][size-1] = '2'

In [ ]:
createMaze()

In [ ]:
print(maze)

[['0', 'X', '0', 'X', 'X', '0', 'X', '0', '0', 'X'], ['0', 'X', '0', '0', 'X', '0', '0', '0', '0', '0'], ['X', '0', '0', '0', 'X', '0', '0', 'X', 'X', 'X'], ['X', '0', '0', '0', '0', '0', 'X', 'X', '0', 'X'], ['0', '0', 'X', '0', '0', '0', 'X', '0', '0', 'X'], ['0', '0', '0', '0', '0', 'X', '0', '0', 'X', 'X'], ['0', '0', 'X', 'X', '0', '0', 'X', '0', '0', 'X'], ['X', 'X', '0', '0', '0', '0', '0', 'X', '0', '0'], ['0', '0', '0', 'X', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', 'X', '0', '0', 'X', '0', '0', '2']]


In [ ]:
def genMaze(size):
  x_data = []
  x_row_list = []

  for row in range(size):
  #get each char we then convert it to string for tensorflow calcs
    for col in maze[row]:
      i = 0
      if(col == 'X'):
        i = -1
      elif(col == '\n'):
        continue
      elif(col == '2'):
        i = 2
      elif(col == '1'):
        i = 1
      x_row_list.append(i)
    #when we have 2500 chars we know we have a 50x50 make it an array and add it to list
  if(len(x_row_list) == 100):
    x_data.append(np.array(x_row_list).reshape(10,10))
    x_row_list = []
  # turn list into numpy array, turn numpy array into catagorical input
  return np.array(x_data)

In [ ]:
def getDir(y_arr):
  max = -9
  idx = 0
  curr = 0
  for i in range(len(y_arr)):
    if y_arr[i] > max:
      idx = i
  
  dir = [0,0,0,0]
  dir[idx] = 1
  
  
  return dir

In [ ]:
new_maze = genMaze(size)

In [ ]:
print(new_maze)

[[[ 0  0  0  0  0  0  0 -1  0 -1]
  [ 0  0  0  0  0  0  0  0  0 -1]
  [ 0  0  0  0  0  0  0  0  0  0]
  [-1  0  0 -1  0  0  0  0 -1 -1]
  [-1  0 -1  0  0 -1  0  0  0 -1]
  [ 0 -1 -1  0 -1  0 -1 -1 -1  0]
  [ 0  0  0  0  0  0 -1  0 -1  0]
  [-1  0  0  0  0  0  0  0  0  0]
  [-1  0  0 -1  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]]


In [ ]:
maze= [['w' for _ in range(size)]for _ in range(size)]
createMaze()
new_maze = genMaze(size)
new_maze[0][0][0] = 1
print(new_maze)

[[[ 1  0  0  0  0  0  0 -1  0 -1]
  [ 0  0  0  0  0  0  0  0  0 -1]
  [ 0  0  0  0  0  0  0  0  0  0]
  [-1  0  0 -1  0  0  0  0 -1 -1]
  [-1  0 -1  0  0 -1  0  0  0 -1]
  [ 0 -1 -1  0 -1  0 -1 -1 -1  0]
  [ 0  0  0  0  0  0 -1  0 -1  0]
  [-1  0  0  0  0  0  0  0  0  0]
  [-1  0  0 -1  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]]


In [ ]:
print(tx[:15])
direct = model.predict(tx[:15])

[[[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  1  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  1  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 ...

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0 -1 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0 -1 ...  0  0  0]
  ...
  [ 0  0  0 ... -1  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0 -1 ...  0  0  0]
  ...
  [ 0  0  0 ... -1 -1  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]]


In [ ]:
print(direct[:15])

[[9.99617100e-01 1.94648155e-08 1.22299156e-04 2.60579371e-04]
 [2.48855300e-04 1.98699240e-07 4.84474526e-07 9.99750435e-01]
 [8.42925947e-05 1.11523896e-05 9.99904156e-01 3.27446685e-07]
 [1.51996464e-05 2.33682940e-07 3.88710859e-08 9.99984622e-01]
 [8.08173352e-16 1.45111380e-11 1.26183644e-04 9.99873757e-01]
 [2.27650574e-14 1.07435771e-09 9.99999762e-01 2.27224731e-07]
 [6.87057300e-09 8.95987541e-05 9.99693274e-01 2.17074077e-04]
 [5.18214269e-13 4.34077644e-12 2.72932346e-04 9.99727070e-01]
 [1.64320070e-18 8.92790819e-09 1.86273886e-04 9.99813735e-01]
 [1.65555849e-31 5.87883582e-22 3.97095638e-12 1.00000000e+00]
 [5.12953883e-21 9.02542624e-12 9.99986172e-01 1.38547884e-05]
 [3.40053593e-12 1.49209100e-05 9.96995926e-01 2.98905442e-03]
 [1.29466172e-26 7.40924674e-20 1.00000000e+00 1.71133081e-08]
 [1.15664745e-36 9.84062071e-25 1.92507196e-06 9.99998093e-01]
 [0.00000000e+00 5.23802014e-25 1.62970679e-10 1.00000000e+00]]


In [ ]:
for i in range(15):
  print(getDir(direct[i]))

[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]


In [ ]:
print(tx[:10])

[[[ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]

 [[ 0  1  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]

 [[ 0  0  0  0  0  0  0  0  0  0]
  [ 0  1  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  